# Two-Stage OSR Runner (Colab / VS Code Colab Kernel)

Pure-Python notebook cells (no `!` or `%` magics) to avoid syntax issues in VS Code.

In [ ]:
import os, sys, subprocess
print('python:', sys.executable)
print('cwd:', os.getcwd())
subprocess.run(['nvidia-smi'], check=False)

In [ ]:
# Optional: mount Drive if running on Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted')
except Exception as e:
    print('Drive mount skipped:', e)

In [ ]:
from pathlib import Path
import os, subprocess

REPO_URL = 'https://github.com/spinelessknave8/FYP_code.git'
REPO_DIR = Path('/content/FYP-code')

if not REPO_DIR.exists():
    subprocess.check_call(['git', 'clone', REPO_URL, str(REPO_DIR)])

os.chdir(REPO_DIR)
print('repo root:', Path.cwd())
print('has config:', (Path('configs/default.yaml')).exists())
print('has src:', (Path('src')).exists())

In [ ]:
import importlib, subprocess, sys

def has(mod):
    try:
        importlib.import_module(mod)
        return True
    except Exception:
        return False

core = [
    ('numpy', 'numpy<2'),
    ('scipy', 'scipy>=1.10'),
    ('PIL', 'pillow>=9.5'),
    ('sklearn', 'scikit-learn>=1.2'),
    ('matplotlib', 'matplotlib>=3.7'),
    ('tqdm', 'tqdm>=4.65'),
    ('yaml', 'pyyaml>=6.0'),
]
missing = [req for mod, req in core if not has(mod)]
if missing:
    print('Installing missing core deps:', missing)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])
else:
    print('Core deps already available')

if not has('torch') or not has('torchvision'):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torch>=2.0', 'torchvision>=0.15'])

import torch, torchvision
print('torch:', torch.__version__, 'torchvision:', torchvision.__version__)

In [ ]:
from pathlib import Path
print('severstal exists:', Path('data/severstal').exists())
print('neu exists:', Path('data/neu').exists())
if not Path('data/severstal').exists() or not Path('data/neu').exists():
    raise RuntimeError('Missing data/severstal or data/neu in runtime')

In [ ]:
# Optional: force GPU in temporary config copy
import yaml
from pathlib import Path
cfg = yaml.safe_load(Path('configs/default.yaml').read_text())
cfg['device'] = 'cuda'
Path('configs/default.colab.yaml').write_text(yaml.safe_dump(cfg, sort_keys=False))
print('wrote configs/default.colab.yaml')

In [ ]:
from src.pipelines.notebook_entrypoints import run_two_stage_stage1, run_split_pipeline
run_two_stage_stage1('configs/default.colab.yaml')
run_split_pipeline('configs/neu_split_a.yaml')
run_split_pipeline('configs/neu_split_b.yaml')
run_split_pipeline('configs/neu_split_c.yaml')

In [ ]:
import json
from pathlib import Path
for split in ['split_a', 'split_b', 'split_c']:
    p = Path('outputs') / split / 'cascade' / 'metrics.json'
    if not p.exists():
        print(split, 'missing metrics')
        continue
    m = json.loads(p.read_text())
    print(split, {
        'tpr_unknown_system': m.get('tpr_unknown_system'),
        'fpr_known_system': m.get('fpr_known_system'),
        'stage1_pass_rate_known': m.get('stage1_pass_rate_known'),
        'stage1_pass_rate_unknown': m.get('stage1_pass_rate_unknown'),
    })